In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer
! pip install -U bitsandbytes

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-iev13p7b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-iev13p7b
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install --upgrade pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
dataset_name = "AISHELL/AISHELL-1"

language = "Chinese"
language_abbr = "zh"
task = "transcribe"

# final model
model_dir = "/content/drive/MyDrive/fyp/AISHELL-1/model/final"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

import torchaudio
import IPython.display as ipd

import whisper
import torchaudio
import torch


In [ ]:
from datasets import load_dataset

dataset = load_dataset("AISHELL/AISHELL-1")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/102 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls drive/MyDrive/fyp/AISHELL-1/data_aishell_transcript_aishell_transcript_v0.8.txt

drive/MyDrive/fyp/AISHELL-1/data_aishell_transcript_aishell_transcript_v0.8.txt


In [ ]:
label_path = "drive/MyDrive/fyp/AISHELL-1/data_aishell_transcript_aishell_transcript_v0.8.txt"

In [ ]:
labels = {}

# Read the transcript file line by line
with open(label_path, 'r') as f:
    for line in f:
        parts = line.strip().split(' ')
        audio_file = parts[0]  # Audio file path (like BAC009S0002W0122.wav)
        transcript = ' '.join(parts[1:])  # Transcript text
        labels[audio_file] = transcript

# Add the labels to the dataset
def add_labels_to_sample(sample):
    # Extract audio file name using '__key__' and add '.wav' extension
    audio_file_name = sample['__key__'].split('/')[-1]

    # Get the corresponding transcript from the labels dictionary
    transcript = labels.get(audio_file_name, None)

    if transcript:
        # Remove spaces from the transcript (if desired)
        transcript_no_spaces = transcript.replace(' ', '')  # This removes all spaces

        sample['transcript'] = transcript_no_spaces
    else:
        print(f"Warning: No transcript found for {audio_file_name}")
        sample['transcript'] = None

    return sample

# Add the labels to the training dataset only
dataset['train'] = dataset['train'].map(add_labels_to_sample)

# Check the result by printing the first sample
print(dataset['train'][0])  # Print the first sample to see the transcript added


{'wav': {'path': 'train/S0002/BAC009S0002W0122.wav', 'array': array([-0.00033569, -0.00054932, -0.00048828, ..., -0.00338745,
       -0.00314331, -0.00350952]), 'sampling_rate': 16000}, '__key__': 'train/S0002/BAC009S0002W0122', '__url__': '/root/.cache/huggingface/hub/datasets--AISHELL--AISHELL-1/snapshots/bbe295d530192a4cd41644b711c9aecd087df653/data_aishell/wav/S0002.tar.gz', 'transcript': '而对楼市成交抑制作用最大的限购'}


In [ ]:
missing_transcripts_count = sum(1 for sample in dataset['train'] if sample['transcript'] is None)

# Print the number of samples with missing transcripts
print(f"Number of samples with missing transcripts: {missing_transcripts_count}")

# Remove samples with missing transcripts
dataset['train'] = dataset['train'].filter(lambda sample: sample['transcript'] is not None)

# Check the result by printing the first sample of the cleaned dataset
print(dataset['train'][0])

Number of samples with missing transcripts: 37
{'wav': {'path': 'train/S0002/BAC009S0002W0122.wav', 'array': array([-0.00033569, -0.00054932, -0.00048828, ..., -0.00338745,
       -0.00314331, -0.00350952]), 'sampling_rate': 16000}, '__key__': 'train/S0002/BAC009S0002W0122', '__url__': '/root/.cache/huggingface/hub/datasets--AISHELL--AISHELL-1/snapshots/bbe295d530192a4cd41644b711c9aecd087df653/data_aishell/wav/S0002.tar.gz', 'transcript': '而对楼市成交抑制作用最大的限购'}


In [ ]:
from datasets import load_dataset

# Example: Load your dataset as a DatasetDict
# dataset = load_dataset('some_dataset')

subset_fraction = 0.25
subset_size = int(len(dataset["train"]) * subset_fraction)

# Subset the 'train' portion of the dataset
train_subset = dataset["train"].shuffle(seed=42).select(range(subset_size))

# Now perform the splits on the subset
train_temp = train_subset.train_test_split(test_size=0.3, seed=42)
test_valid = train_temp["test"].train_test_split(test_size=0.5, seed=42)

train_dataset = train_temp["train"]
val_dataset = test_valid["train"]
test_dataset = test_valid["test"]

print(f"Training: {len(train_dataset)}, Validation: {len(val_dataset)}, Testing: {len(test_dataset)}")


Training: 6068, Validation: 1300, Testing: 1301


In [ ]:
from datasets import Audio, DatasetDict

dataset = DatasetDict()

dataset["train"] = train_dataset
dataset["validation"] = val_dataset
dataset["test"] = test_dataset

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_dir, language=language, task=task)

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_dir, language=language, task=task)

In [ ]:
def prepare_dataset(batch):
    try:
        audio = batch["wav"]
        features = feature_extractor(
            audio["array"],
            sampling_rate=audio["sampling_rate"]
        )
        batch["input_features"] = features.input_features[0]
        batch["labels"] = tokenizer(batch["transcript"]).input_ids
    except Exception as e:
        print(f"Error processing sample: {e}")
        batch["input_features"] = None  # Mark for filtering later
    return batch

tokenized_dataset = dataset.map(prepare_dataset)

Map:   0%|          | 0/6068 [00:00<?, ? examples/s]

Map:   0%|          | 0/1300 [00:00<?, ? examples/s]

Map:   0%|          | 0/1301 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['wav', '__key__', '__url__', 'transcript', 'input_features', 'labels'],
        num_rows: 6068
    })
    validation: Dataset({
        features: ['wav', '__key__', '__url__', 'transcript', 'input_features', 'labels'],
        num_rows: 1300
    })
    test: Dataset({
        features: ['wav', '__key__', '__url__', 'transcript', 'input_features', 'labels'],
        num_rows: 1301
    })
})

In [ ]:
model_dir_1 = "/content/drive/MyDrive/fyp/AISHELL-1/model/final"
model_dir_2 = "openai/whisper-large-v2"


model_1 = WhisperForConditionalGeneration.from_pretrained(model_dir_1)
processor_1 = WhisperProcessor.from_pretrained(model_dir_1, language="zh", task="transcribe")
model_1.to(device)
model_1.eval()

model_2 = WhisperForConditionalGeneration.from_pretrained(model_dir_2)
processor_2 = WhisperProcessor.from_pretrained(model_dir_2, language="zh", task="transcribe")
model_2.to(device)
model_2.eval()


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias

In [ ]:
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from jiwer import wer  # For Word Error Rate
import numpy as np

# Function to compute Character Error Rate (CER)
def cer(prediction, reference):
    """
    Calculate Character Error Rate (CER) between the predicted text and the reference.
    CER = (Insertions + Deletions + Substitutions) / (Total Characters in Reference)
    """
    prediction = prediction.replace(" ", "")  # Remove spaces for CER calculation
    reference = reference.replace(" ", "")  # Remove spaces for CER calculation
    # Compute the Levenshtein distance between the predicted and reference strings
    from editdistance import eval as edit_distance
    dist = edit_distance(prediction, reference)
    return dist / len(reference) if len(reference) > 0 else 0

# Define a function to process and transcribe a single sample
def transcribe_sample(model, processor, audio_array, sampling_rate):
    # Resample to 16kHz if needed
    if sampling_rate != 16000:
        audio_array = librosa.resample(audio_array.astype("float32"), orig_sr=sampling_rate, target_sr=16000)

    # Preprocess the input
    inputs = processor(audio_array, sampling_rate=16000, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate tokens
    with torch.no_grad():
        predicted_ids = model.generate(inputs["input_features"])

    # Decode predicted tokens to text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    return transcription

# Initialize variables for total CER and WER
total_cer_1 = 0
total_cer_2 = 0
total_wer_1 = 0
total_wer_2 = 0
total_samples = len(tokenized_dataset["test"])

# Loop through the test dataset and compare transcriptions from both models
for idx, sample in enumerate(tokenized_dataset["test"]):
    audio_array = sample["wav"]["array"]
    sampling_rate = sample["wav"]["sampling_rate"]
    true_label = sample["transcript"]

    # Get transcriptions from both models
    transcription_1 = transcribe_sample(model_1, processor_1, audio_array, sampling_rate)
    transcription_2 = transcribe_sample(model_2, processor_2, audio_array, sampling_rate)

    # Calculate CER for both models
    cer_1 = cer(transcription_1, true_label)
    cer_2 = cer(transcription_2, true_label)

    # Calculate WER for both models
    wer_1 = wer(true_label, transcription_1)
    wer_2 = wer(true_label, transcription_2)

    # Accumulate CER and WER values
    total_cer_1 += cer_1
    total_cer_2 += cer_2
    total_wer_1 += wer_1
    total_wer_2 += wer_2

    # Optionally print each sample's results
    print(f"Sample {idx + 1}")
    print(f"True Label: {true_label}")
    print(f"Model 1 (whisper-trained-lora) Transcription: {transcription_1}")
    print(f"Model 2 (whisper-large-v2) Transcription: {transcription_2}")
    print(f"Model 1 CER: {cer_1:.4f}  WER: {wer_1:.4f}")
    print(f"Model 2 CER: {cer_2:.4f}  WER: {wer_2:.4f}")
    print("-" * 50)  # Separator between samples

# Calculate average CER and WER across all samples
average_cer_1 = total_cer_1 / total_samples
average_cer_2 = total_cer_2 / total_samples
average_wer_1 = total_wer_1 / total_samples
average_wer_2 = total_wer_2 / total_samples

# Output average CER and WER for both models
print(f"Average CER for Model 1 (whisper-trained-lora): {average_cer_1:.4f}")
print(f"Average CER for Model 2 (whisper-large-v2): {average_cer_2:.4f}")
print(f"Average WER for Model 1 (whisper-trained-lora): {average_wer_1:.4f}")
print(f"Average WER for Model 2 (whisper-large-v2): {average_wer_2:.4f}")


Streaming output truncated to the last 5000 lines.
Model 1 (whisper-trained-lora) Transcription: 地址山行县米泽市湾支嫩伊丁木四百一十三
Model 2 (whisper-large-v2) Transcription: 地址山形縣米澤市灣,知嫩,伊丁木413
Model 1 CER: 0.3158  WER: 1.0000
Model 2 CER: 0.6842  WER: 1.0000
--------------------------------------------------
Sample 589
True Label: 污水处理费低于处理成本状况将得到解决
Model 1 (whisper-trained-lora) Transcription: 污水处理费低于处理成本的状况将得到解决
Model 2 (whisper-large-v2) Transcription: 污水处理费低于处理成本的状况将得到解决
Model 1 CER: 0.0556  WER: 1.0000
Model 2 CER: 0.0556  WER: 1.0000
--------------------------------------------------
Sample 590
True Label: 一直寻求重组万好万家公布重组预案公布
Model 1 (whisper-trained-lora) Transcription: 一直寻求重组万好万佳公布重组预案公布
Model 2 (whisper-large-v2) Transcription: 一直尋求重組萬好萬佳公佈重組預案公佈
Model 1 CER: 0.0556  WER: 1.0000
Model 2 CER: 0.5000  WER: 1.0000
--------------------------------------------------
Sample 591
True Label: 有其历史性和现实性
Model 1 (whisper-trained-lora) Transcription: 有其历史性和现实性
Model 2 (whisper-large-v2) Transcription: 有